# Import Modules and Inputs

In [1]:
import ee, geemap, time

Intialize Geemap

In [2]:
Map = geemap.Map(center=(10.41, 37.62), zoom=11)

jedeb = ee.FeatureCollection("projects/ee-yilkalg3/assets/Jedeb_Watershed")
#Map.addLayer(jedeb, {}, 'Jedeb Watershed')

Set Dates

In [3]:
start = '2021-12-01'
end = '2022-04-30'
season = ee.Filter.date(start,end)

Import Training Dataset

In [4]:
points = ee.FeatureCollection("projects/ee-yilkalg3/assets/TrainingJ")
print(points.size().getInfo())

896


Import Validation Dataset

In [5]:
validationGcp = ee.FeatureCollection("projects/ee-yilkalg3/assets/ValidationJ")
print(validationGcp.size().getInfo())

225


In [6]:
#validationGcp.first().getInfo()

# Sentinel 2 only

In [7]:
# Load Median Sentinel 2 Reflectance 
def maskS2clouds(image):
  qa = image.select('QA60')

  # Bits 10 and 11 are clouds and cirrus, respectively.
  cloudBitMask = 1 << 10
  cirrusBitMask = 1 << 11

  # Both flags should be set to zero, indicating clear conditions.
  mask = qa.bitwiseAnd(cloudBitMask).eq(0) \
      .And(qa.bitwiseAnd(cirrusBitMask).eq(0))

  return image.updateMask(mask).divide(10000)

# Filter the first Sentinel 2 image of the month
sentinel2 = ee.ImageCollection('COPERNICUS/S2_SR')\
    .filterBounds(jedeb)\
    .filter(season)\
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',20))\
    .map(maskS2clouds)
 
# Median image
s2median = ee.Image(sentinel2.median()).clip(jedeb)
s2median.bandNames().getInfo()

visualization = {
  'min': 0.0,
  'max': 0.3,
  'bands': ['B4', 'B3', 'B2'],
}

Map.addLayer(s2median, visualization, 'RGB')

## Train the classifier

In [8]:
# Use these bands for prediction (excluding red edge bands).
bands = ['B2', 'B3', 'B4', 'B5', 'B6','B7','B8', 'B8A', 'B11', 'B12']

# This property of the table stores the land cover labels.
label = 'Cover'

# Overlay the points on the imagery to get training.
sample = s2median.select(bands).sampleRegions(
    **{'collection': points, 'properties': [label], 'scale': 10}
)

classifier = ee.Classifier.smileGradientTreeBoost(128).train(sample, label, bands)

## Classify the Image

In [9]:
# Classify the image with the same bands used for training 
classified_RF1 = s2median.select(bands).classify(classifier)

## Accuracy assessment

Training Accuracy

In [10]:
train_accuracy = classifier.confusionMatrix()
#train_accuracy.getInfo()

In [11]:
#train_accuracy.accuracy().getInfo()

In [12]:
#train_accuracy.kappa().getInfo()

In [13]:
#train_accuracy.producersAccuracy().getInfo()

In [14]:
#train_accuracy.consumersAccuracy().getInfo()

Validation Accuracy

In [15]:
# This property of the table stores the land cover labels.
label = 'Cover'

band = ['classification']
# Overlay the points on the imagery to get training.
test = classified_RF1.select(band).sampleRegions(
    **{'collection': validationGcp, 'properties':[label],'scale': 10}
)

In [16]:
test_accuracy = test.errorMatrix('Cover', 'classification')
#test_accuracy.getInfo()

In [17]:
test_accuracy.accuracy().getInfo()

0.8088888888888889

In [18]:
test_accuracy.kappa().getInfo()

0.7559283551967709

In [19]:
test_accuracy.producersAccuracy().getInfo()

[[0],
 [1],
 [0.8],
 [0.6764705882352942],
 [0.8181818181818182],
 [0.8490566037735849],
 [1],
 [0.7777777777777778]]

# Sentinel 1 and Sentinel 2

In [20]:
# Import Sentinel-1 collection
sentinel1 =  ee.ImageCollection('COPERNICUS/S1_GRD')\
                .filterBounds(jedeb)\
                .filter(season)\
                .filter(ee.Filter.eq('instrumentMode', 'IW'))\
                .filter(ee.Filter.eq('resolution_meters', 10)) 
                
# Number of tiles in the season
sentinel1 = sentinel1.sort('system:time_start')
sentinel1.size().getInfo()

13

In [21]:
# the acquisition times in the collection, formatted with Python's time module
acq_times = sentinel1.aggregate_array('system:time_start').getInfo()
[time.strftime('%x', time.gmtime(acq_time/1000)) for acq_time in acq_times]

['12/02/21',
 '12/14/21',
 '12/26/21',
 '01/07/22',
 '01/19/22',
 '01/31/22',
 '02/12/22',
 '02/24/22',
 '03/08/22',
 '03/20/22',
 '04/01/22',
 '04/13/22',
 '04/25/22']

In [22]:
# using monthly mean backscatter 

s1Dec = ee.Image(sentinel1.filterDate("2021-12-01","2021-12-31").mean()).select(["VV","VH"],["VVdec","VHdec"])
s1Jan = ee.Image(sentinel1.filterDate("2022-01-01","2022-01-31").mean()).select(["VV","VH"],["VVjan","VHjan"]);
s1Feb = ee.Image(sentinel1.filterDate("2022-02-01","2022-02-28").mean()).select(["VV","VH"],["VVfeb","VHfeb"])
s1Mar = ee.Image(sentinel1.filterDate("2022-03-01","2022-03-31").mean()).select(["VV","VH"],["VVmar","VHmar"])
s1Apr = ee.Image(sentinel1.filterDate("2022-04-01","2022-04-30").mean()).select(["VV","VH"],["VVapr","VHapr"])

sentinel1mean = s1Dec.addBands(s1Jan)\
                 .addBands(s1Feb)\
                 .addBands(s1Mar)\
                 .addBands(s1Apr)

#print(sentinel1mean.getInfo())

In [23]:
composite = ee.Image.cat([(s2median),(sentinel1mean)])
sentinel = composite.clip(jedeb)
#print(Sentinel.getInfo())

## Train the Classifier

In [24]:
bands = ['B2', 'B3', 'B4','B8', 'B8A', 'B11', 'B12', 'VVdec', 'VHdec', 'VVjan', 'VHjan', 'VVfeb', 'VHfeb', 'VVmar', 'VHmar', 'VVapr', 'VHapr']
# This property of the table stores the land cover labels.
label = 'Cover'

# Overlay the points on the imagery to get training.
sample = sentinel.select(bands).sampleRegions(
    **{'collection': points, 'properties': [label], 'scale': 10}
)

In [25]:
classifier = ee.Classifier.smileGradientTreeBoost(128).train(sample, label, bands)

## Classify the Image

In [26]:
# Classify the image with the same bands used for training.
classified_RF2 = sentinel.select(bands).classify(classifier)

## Accuracy assessment

Training Accuracy

In [27]:
train_accuracy = classifier.confusionMatrix()
#train_accuracy.getInfo()

In [28]:
#train_accuracy.accuracy().getInfo()

In [29]:
#train_accuracy.kappa().getInfo()

In [30]:
#train_accuracy.producersAccuracy().getInfo()

In [31]:
#train_accuracy.consumersAccuracy().getInfo()

Validation Accuarcy

In [32]:
# This property of the table stores the land cover labels.
label = 'Cover'

band = ['classification']
# Overlay the points on the imagery to get training.
test = classified_RF2.select(band).sampleRegions(
    **{'collection': validationGcp, 'properties':[label],'scale': 10}
)

In [33]:
test_accuracy = test.errorMatrix('Cover', 'classification')
#test_accuracy.getInfo()

In [34]:
test_accuracy.accuracy().getInfo()

0.8888888888888888

In [35]:
test_accuracy.kappa().getInfo()

0.8579904064630143

In [36]:
test_accuracy.producersAccuracy().getInfo()

[[0],
 [1],
 [0.8714285714285714],
 [0.7647058823529411],
 [0.9545454545454546],
 [0.9245283018867925],
 [1],
 [0.7777777777777778]]

In [37]:
test_accuracy.consumersAccuracy().getInfo()

[[0,
  1,
  0.9384615384615385,
  0.9629629629629629,
  0.8936170212765957,
  0.8032786885245902,
  0.9230769230769231,
  0.7777777777777778]]

# Improving Classification 

## Vegetation Indices 

EVI

In [38]:
def getEVI(image):
    # Compute the EVI using an expression.
    EVI = image.expression(
        '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))', {
            'NIR': image.select('B8'),
            'RED': image.select('B4'),
            'BLUE': image.select('B2')
        }).rename("EVI")

    image = image.addBands(EVI)

    return(image)

evi = sentinel2.map(getEVI).median().clip(jedeb).select('EVI')

print(evi.getInfo())

{'type': 'Image', 'bands': [{'id': 'EVI', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}]}


NDVI

In [39]:
# Calculate NDVI
def getNDVI(image):
    # Compute the EVI using an expression.
    NDVI = image.expression(
        '((NIR - RED) / (NIR + RED ))', {
            'NIR': image.select('B8'),
            'RED': image.select('B4'),
        }).rename("NDVI")

    image = image.addBands(NDVI)

    return(image)

ndvi = sentinel2.map(getNDVI).median().clip(jedeb).select('NDVI')

print(ndvi.getInfo())

{'type': 'Image', 'bands': [{'id': 'NDVI', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}]}


NDBI

In [40]:
# Calculate NDBI
def getNDBI(image):
    # Compute the EVI using an expression.
    NDBI = image.expression(
        '((SWIR - NIR) / (SWIR + NIR ))', {
            'SWIR': image.select('B11'),
            'NIR': image.select('B8'),
        }).rename("NDBI")

    image = image.addBands(NDBI)

    return(image)

ndbi = sentinel2.map(getNDBI).median().clip(jedeb).select('NDBI')

print(ndbi.getInfo())

{'type': 'Image', 'bands': [{'id': 'NDBI', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}]}


Slope

## Auxillary Data

In [41]:
dem = ee.Image('NASA/NASADEM_HGT/001').select('elevation').clip(jedeb)

In [42]:
slope = ee.Terrain.slope(dem)
Map.addLayer(slope, {min: 0, max: 89.99}, 'Slope')
print(slope.getInfo())

{'type': 'Image', 'bands': [{'id': 'slope', 'data_type': {'type': 'PixelType', 'precision': 'float', 'min': 0, 'max': 90}, 'crs': 'EPSG:4326', 'crs_transform': [0.0002777777777777778, 0, -179.0001388888889, 0, -0.0002777777777777778, 61.00013888888889]}]}


Composite

In [43]:
composite = ee.Image.cat([(sentinel),(evi),(ndvi),(ndbi),(dem)])

sentinel_vi = composite.clip(jedeb)

## Train the classifier 

In [44]:
bands = ['B2', 'B3', 'B4','B8', 'B8A', 'B11', 'B12',\
         'VVdec', 'VHdec', 'VVjan', 'VHjan', 'VVfeb', 'VHfeb', 'VVmar', 'VHmar', 'VVapr', 'VHapr',\
         'EVI','NDBI','elevation']
# This property of the table stores the land cover labels.
label = 'Cover'

# Overlay the points on the imagery to get training.
sample = sentinel_vi.select(bands).sampleRegions(
    **{'collection': points, 'properties': [label], 'scale': 10}
)

In [45]:
# Train a 50-tree gradient boosting classifier from the training sample.
classifier = ee.Classifier.smileGradientTreeBoost(128).train(sample, label, bands)

## Classify the Image

In [46]:
# Classify the image with the same bands used for training.
classified_RF3 = sentinel_vi.select(bands).classify(classifier)

## Accuracy Assessment

Training Accuarcy

In [47]:
train_accuracy = classifier.confusionMatrix()
#train_accuracy.getInfo()

In [48]:
#train_accuracy.accuracy().getInfo()

In [49]:
#train_accuracy.kappa().getInfo()

In [50]:
#train_accuracy.producersAccuracy().getInfo()

In [51]:
#train_accuracy.consumersAccuracy().getInfo()

Validation Accuarcy

In [52]:
# This property of the table stores the land cover labels.
label = 'Cover'

band = ['classification']
# Overlay the points on the imagery to get training.
test = classified_RF3.select(band).sampleRegions(
    **{'collection': validationGcp, 'properties':[label],'scale': 10}
)

In [53]:
test_accuracy = test.errorMatrix('Cover', 'classification')
#test_accuracy.getInfo()

In [54]:
test_accuracy.accuracy().getInfo()

0.8977777777777778

In [55]:
test_accuracy.kappa().getInfo()

0.869043702710226

In [56]:
test_accuracy.producersAccuracy().getInfo()

[[0],
 [1],
 [0.9142857142857143],
 [0.7647058823529411],
 [0.9545454545454546],
 [0.9056603773584906],
 [1],
 [0.7777777777777778]]

In [57]:
test_accuracy.consumersAccuracy().getInfo()

[[0,
  1,
  0.927536231884058,
  0.9285714285714286,
  0.8936170212765957,
  0.8421052631578947,
  0.9230769230769231,
  0.875]]

# Visualize Results

In [58]:
# Define a palette for the classification.
Palette = [
    '419BDF', #water
    '397D49', #trees
    '88B053', #grass
    '6efc6a', #irrigation
    'E49635', #crops
    'C4281B', #built
    'A59B8F', #bare
]
Map.addLayer(classified_RF3, {'palette': Palette, 'min': 1, 'max': 7}, 'classification')
Map

Map(center=[10.41, 37.62], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(childre…

# Calculate Statistics

In [ ]:
# Calculate total area of a single class (eq(*))     * is the class number 
area = classified_RF1.eq(4).multiply(ee.Image.pixelArea()).reduceRegion(**{
  'reducer': ee.Reducer.sum(),
  'geometry': jedeb.geometry(),
  'scale': 10,
  'maxPixels': 1e9
})

area.getInfo()

In [ ]:
# Calculate total area of a single class (eq(*))     * is the class number 
area = classified_RF2.eq(4).multiply(ee.Image.pixelArea()).reduceRegion(**{
  'reducer': ee.Reducer.sum(),
  'geometry': jedeb.geometry(),
  'scale': 10,
  'maxPixels': 1e9
})

print(area.getInfo(),'Area in ha')

In [ ]:
# Calculate total area of a single class (eq(*))     * is the class number 
area = classified_RF3.eq(4).multiply(ee.Image.pixelArea()).reduceRegion(**{
  'reducer': ee.Reducer.sum(),
  'geometry': jedeb.geometry(),
  'scale': 10,
  'maxPixels': 1e9
})

area.getInfo()